In [1]:
from langchain.llms import OpenAI 
from langchain.chat_models import ChatOpenAI  
from langchain.callbacks import get_openai_callback
from langchain.schema.messages import HumanMessage, SystemMessage
import os
import json
import time
import pandas as pd

In [2]:
script_path = os.getcwd()
print(script_path)
parent_path = os.path.dirname(os.path.dirname(script_path))
print(parent_path)
data_path = os.path.join(parent_path,"outbreak","COVID19","outbreak_announcements","San Diego County","data","archive")
filelist = os.listdir(data_path)
print(len(filelist))
print(filelist[0])
testlist = filelist[130:140]
print(testlist)

C:\Users\gtsueng\Anaconda3\envs\nde\openai_topic_classification
C:\Users\gtsueng\Anaconda3\envs
1318
USA_US.CA_06073_20200102.09.02.07.json
['USA_US.CA_06073_20200413.18.32.10.json', 'USA_US.CA_06073_20200414.14.06.40.json', 'USA_US.CA_06073_20200414.17.02.02.json', 'USA_US.CA_06073_20200415.17.31.59.json', 'USA_US.CA_06073_20200416.16.14.50.json', 'USA_US.CA_06073_20200417.10.00.07.json', 'USA_US.CA_06073_20200417.18.09.01.json', 'USA_US.CA_06073_20200420.19.21.15.json', 'USA_US.CA_06073_20200421.08.40.47.json', 'USA_US.CA_06073_20200421.16.16.46.json']


In [11]:
def get_content_from_file(file_path):
    """Read and return content from a file."""
    try:
        with open(file_path, 'rb') as file:
            return file.read().decode("UTF-8").strip()
    except FileNotFoundError:
        raise FileNotFoundError(f"No such file: '{file_path}'")
    except Exception as e:
        raise e

In [18]:
script_path = os.getcwd()
print(script_path)

# Path to the API key and prompt text files
api_key_file_path = os.path.join(script_path,"key.txt")
prompt_file_path = os.path.join(script_path,"prompt7.txt")

# Read API key and prompt from files
api_key = get_content_from_file(api_key_file_path)
prompt = get_content_from_file(prompt_file_path)

print("===========PROMPT: ==========\n"+prompt+"\n=============\n")

C:\Users\gtsueng\Anaconda3\envs\nde\openai_topic_classification
===========PROMPT: ==========
I have a description associated with a scientific dataset quoted here: 

"Asia Pacific ICEMR Cambodian Cross-sectional
Background: In Cambodia, significant achievements have been made over the past decade in reducing malaria-related morbidity and mortality. However, transmission is endemic in 21 of 25 provinces and it is estimated that 58% of the population, or approximately 8.6 million people, continue to live in malaria at-risk areas. The incidence of clinical cases is highest in the north-eastern part of the country, with the highest risk in forest or forest fringe areas of the country, usually found along the borders. It is therefore very important to i) characterize region specific demographic and geographic determinants of (especially asymptomatic) malaria infection ii) determine the relationship between different measures of malaria infection and exposure and iii) implement malaria surv

In [12]:
#prompt = "Please describe datasets used in the academic article titled, 'Similarities and differences between the ‘cytokine storms’ in acute dengue and COVID-19' by Shashika Dayarathna et. al."

In [19]:
messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(content=prompt)
]

chat = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    openai_api_key = api_key
)

In [20]:
with get_openai_callback() as cb:
    chat.invoke(messages)
    for chunk in chat.stream(messages):
        print(chunk.content, end="", flush=True)
    print("\n===========CALLBACK: ==========\n")
    print(cb)
    print("\n=============\n")

- qPCR (quantitative Polymerase Chain Reaction)
- Light microscopy
- Antibody testing
- Genetic analysis for Plasmodium spp. infections
- Molecular marker analysis for drug resistance
- G6PD deficiency testing
- CYP 2D6 polymorphism analysis
===========CALLBACK: ==========

Tokens Used: 1015
	Prompt Tokens: 968
	Completion Tokens: 47
Successful Requests: 1
Total Cost (USD): $0.001546




In [58]:
promptstart = "I have metadata associated with a public service announcement quoted here:\n"
prompt_file_path = "prompt4.txt"
promptend = prompt = get_content_from_file(prompt_file_path)
resultlist = []
for eachitem in testlist:
    with open(os.path.join(data_path,eachitem),"rb") as infile:
        article = json.load(infile)
        article_info = promptstart+"\n"+article['name']+"\n"+article["articleContent"]+"\n"+promptend
    print(article_info)
    print("================")
    messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(content=article_info)
    ]

    chat = ChatOpenAI(
        model_name='gpt-3.5-turbo',
        openai_api_key = api_key
    )
    gpt_output = ''
    with get_openai_callback() as cb:
        chat.invoke(messages)
        for chunk in chat.stream(messages):
            gpt_output += chunk.content
    resultlist.append({"_id": article["_id"],"response":gpt_output,"callback":cb, 
                        "title":article["name"],"text":article["articleContent"]})
        
    time.sleep(0.5)
    
        



I have metadata associated with a public service announcement quoted here:

2 More COVID-19 Deaths Reported; Cases Increase by 43

Two additional COVID-19 deaths have been reported, bringing the region’s total to 47, the County Health and Human Services Agency announced today.
The two new deaths are a woman in her late 90s and a 100-year-old woman. Both had underlying medical conditions.
“COVID-19 significantly impacts our older adults and people with underlying medical conditions, such as heart disease, chronic lung disease, asthma and diabetes, as well as compromised immune systems,” said Wilma Wooten, M.D., M.P.H., County public health officer. “Our vulnerable populations matter and we want to keep them safe.
That is why we must continue to stay home and take other preventive measures to protect those who are most vulnerable.”
The age, gender and race and ethnic breakdown of the 47 COVID-19 deaths reported is:
Age:

80 and older: 24
70 to 79: 13
60 to 69: 4
50 to 59: 4
20 to 29: 2



I have metadata associated with a public service announcement quoted here:

7 More COVID-19 Deaths Reported, Cases Exceed 2,000

 
The ages of the people who died range from 64 to 100 years. Four are men and three are women and all had underlying medical conditions.
“Our sincere sympathies and condolences go out to the family and friends of all the people who have died because of COVID-19,” said Wilma Wooten, M.D., M.P.H., County public health officer. “The best way to protect those at higher risk of complications from COVID-19 is by staying at your regular home and wearing a face covering when in public.”
The age, gender and race and ethnic breakdown of the local COVID-19 deaths are:
Age:

80 and older: 32
70 to 79: 15
60 to 69: 6
50 to 59: 4
40 to 49: 1
20 to 29: 2

Gender:

34 Men
26 Women

Race and Ethnicity:

24 White
17 Hispanic
4 Asian
1 Multiple races
14 Unknown

County COVID-19 Cases Now at 2,012
Nearly 28,000 tests have been administered to San Diegans. To date, 2,012 people 

I have metadata associated with a public service announcement quoted here:

7 New COVID-19 Deaths Reported; Cases Increase to 2,158

The number of local COVID-19 deaths went up by seven, bringing the region’s total to 70, the County Health and Human Services Agency announced today.
The ages of the seven people who died range from 58 to 95 years. Four are men and three are women and all had existing medical conditions.
“Community spread continues to occur in San Diego County and, unfortunately, more COVID-19 deaths are being reported,” said Wilma Wooten, M.D., M.P.H., County public health officer. “The best way to prevent contracting COVID-19 is to stay home and follow all the other public health recommendations.”
The great majority of the people who have died from COVID-19 are 70 and older. The youngest were two people in their 20s. Of those who have died, 39 were men and 31 were women. Age, gender and ethnic and race breakdowns for deaths and cases are posted at coronavirus-sd.com.
Co

In [59]:
print(resultlist)

[{'_id': 'USA_US.CA_06073_20200413.18.32.10', 'response': '1. masks\n2. stay at home\n3. physical distance', 'callback': Tokens Used: 951
	Prompt Tokens: 940
	Completion Tokens: 11
Successful Requests: 1
Total Cost (USD): $0.0014320000000000001, 'title': '2 More COVID-19 Deaths Reported; Cases Increase by 43', 'text': '\nTwo additional COVID-19 deaths have been reported, bringing the region’s total to 47, the County Health and Human Services Agency announced today.\nThe two new deaths are a woman in her late 90s and a 100-year-old woman. Both had underlying medical conditions.\n“COVID-19 significantly impacts our older adults and people with underlying medical conditions, such as heart disease, chronic lung disease, asthma and diabetes, as well as compromised immune systems,” said Wilma Wooten, M.D., M.P.H., County public health officer. “Our vulnerable populations matter and we want to keep them safe.\nThat is why we must continue to stay home and take other preventive measures to pro

In [55]:
resultdf = pd.DataFrame(resultlist)
print(resultdf.head(n=2))

                                 _id                              response  \
0  USA_US.CA_06073_20200413.18.32.10  stay at home, social distance, masks   

                                            callback  \
0  Tokens Used: 949\n\tPrompt Tokens: 940\n\tComp...   

                                               title  \
0  2 More COVID-19 Deaths Reported; Cases Increas...   

                                                text  
0  \nTwo additional COVID-19 deaths have been rep...  


In [60]:
resultdf.to_csv("results.csv",header=True)

In [5]:
resultlist = []
for eachitem in filelist:
    with open(os.path.join(data_path,eachitem),"rb") as infile:
        article = json.load(infile)
        resultlist.append(article)

resultdf = pd.DataFrame(resultlist)

In [7]:
print(resultdf.head(n=2))
resultdf.to_csv('example_data.csv',header=True)

                 @type                                           @context  \
0  SpecialAnnouncement  {'schema': 'http://schema.org/', 'rdf': 'http:...   
1  SpecialAnnouncement  {'schema': 'http://schema.org/', 'rdf': 'http:...   

                                                name  \
0  2 Flu Deaths and Nearly 1,400 Cases Reported L...   
1    Top 10 Library Book and Movie Checkouts of 2019   

                                         description  \
0  Two additional flu deaths were reported in the...   
1  The beginning of the new year is the perfect t...   

               datePublished  \
0  2020-01-02T09:02:07-08:00   
1  2020-01-02T10:10:06-08:00   

                                                 url  \
0  https://www.countynewscenter.com/2-flu-deaths-...   
1  https://www.countynewscenter.com/top-10-librar...   

                                            keywords  \
0  [health, flu deaths, flu vaccine, influenza de...   
1       [library, movies, reading, top 10 checkouts]

In [8]:
clean = resultdf[['name','description','url','datePublished','articleContent']].copy()
clean.to_csv('clean_data.csv', header=True)